In [11]:
import os
WORK = os.environ["WORK"]
PROJ_DIR = f'{WORK}/ADA_Project'
os.chdir(PROJ_DIR)
print(os.getcwd())

import sys
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/DeepFAMS')

/lustre/work/chaselab/malyetama/ADA_Project


In [12]:
from glob import glob
import numpy as np
import PIL.Image
from tqdm import tqdm
from pathlib import Path
import urllib.request
import tarfile
import zipfile
import subprocess

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import tensorflow as tf
tf.compat.v1.enable_eager_execution()

import DeepFAMS

In [13]:
RAW_IMGS_DIR, RESIZED_IMGS_DIR, DATA_CUSTOM_DIR, TRAIN_RUNS_DIR = DeepFAMS.utils.return_dirs(
    PROJ_DIR, 'gen1_pokemon')

In [27]:
! rm -rf /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_custom

In [ ]:
# ! wget 'https://unomaha.box.com/shared/static/b2quhszs8d7qfh40nnt2sojcztoxmdop.zip' -O datasets/gen1_pokemon.zip

In [ ]:
# import zipfile
# with zipfile.ZipFile('datasets/gen1_pokemon.zip', 'r') as zip_ref:
#     zip_ref.extractall(RAW_IMGS_DIR)

In [14]:
raw_imgs = glob(f'{RAW_IMGS_DIR}/**/**/*')
print(len(raw_imgs))

10842


In [15]:
# for x in tqdm(raw_imgs):
#     if Path(x).suffix.lower() not in ['.jpg', '.jpeg', '.png']:
#         continue
#     else:
#         DeepFAMS.preprocessing.resize_imgs(x, (256, 256), RESIZED_IMGS_DIR)

In [16]:
print(f'Raw: {len(raw_imgs)}, Resized: {len(glob(f"{RESIZED_IMGS_DIR}/*"))}')

Raw: 10842, Resized: 9950


In [30]:
resized_imgs = glob(f"{RESIZED_IMGS_DIR}/*")
for img in resized_imgs:
    im = PIL.Image.open(img)
    if im.size != (256, 256):
        print(im.size, img)

In [31]:
DeepFAMS.preprocessing.tf_record_exporter(
    tfrecord_dir=DATA_CUSTOM_DIR, image_dir=RESIZED_IMGS_DIR, shuffle=1)

Loading images from "/work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized"
Creating dataset "/work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_custom"
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/e803276afa214ad0adc8bb54ca2fa13b.jpg
10 / 9950
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/00000003.png
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/8c828f0c613f4feda674bf2eb8ca5d2d.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/8d18bd13c4a744129b1c22911ec932b6.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/2447ee218b3c4f5f976fa083cb0a42c5.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/2826f40da31b4caa80c0fef7e95f5d1d.jpg

axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/67e860cff57c40d6bd0b303db4a4f04f.jpg
1120 / 9950
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/00000222.png
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/a6497efd977c4f3d9dcb7b2a54d6f690.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/d1633622c82a434ebff07fb2ac5e6701.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/ea9ad5fd283a4c3586ea70fed9b8ce1c.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/e30256b4699446b7b9b242d42375b90a.jpg
axes don't match array
Failed to export /work/chaselab/malyetama/ADA_Project/datasets/gen1_pokemon_resized/4a1628377cab40b094012d3852b346a1.jpg
axes don't match array
Fail

KeyboardInterrupt: 

In [ ]:
# DeepFAMS.utils.executePopen(
# f'''#!/bin/bash
# module load anaconda
# module load compiler/gcc/4.7
# module load cuda
# $WORK/.conda/envs/ada-env/bin/python $WORK/ADA_Project/StyleGAN2-ada__source_code/train.py \
# --outdir={TRAIN_RUNS_DIR} \
# --gpus=2 \
# --data={DATA_CUSTOM_DIR} \
# --snap=1 \
# --kimg=1''', PROJ_DIR
# )

In [8]:
for num in range(-1, -10, -1):
    files = DeepFAMS.utils.last_snap(num, TRAIN_RUNS_DIR)
    if files != []:
        break

latest_snap = sorted(files)[-1]
print(latest_snap)

/work/chaselab/malyetama/ADA_Project/training_runs/gen1_pokemon_training-runs/00008-gen1_pokemon_custom-auto2-resumecustom/network-snapshot-000110.pkl


In [ ]:
DeepFAMS.utils.execute('nvidia-smi')

In [ ]:
run_desc, training_options = DeepFAMS.setup_training_options(
    gpus       = 2,
    snap       = 1,
    data       = DATA_CUSTOM_DIR,
    resume     = latest_snap
)

In [ ]:
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=True, run_desc=run_desc, training_options=training_options)

In [ ]:
tf.compat.v1.disable_eager_execution()
DeepFAMS.RunTraining(outdir=TRAIN_RUNS_DIR, seed=1000,
             dry_run=False, run_desc=run_desc, training_options=training_options)